In [26]:
import sys
import os
import geopandas as gpd
import pandas as pd

# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.calculate import calc_leistungskm
from src.import_gpx import import_gpx

from src.maps import generate_elevation_plot 
from src.MICHI import export_to_pdf


In [9]:
gpx = import_gpx("C:/Users/aebim/Documents/02_Ausbildung/Studium/04_Semester/4040_Geoprogrammieren_II/GrSchmeterling_Hackathon/data/7-gipfel-tour-flumserberg.gpx")

c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\io\file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [10]:
gpx.head()


,segment_id,von_pkt_name,von_pkt_geom,bis_pkt_name,bis_pkt_geom,segment_geom
0,1,Startpunkt,POINT Z (2737832.731 1214786.689 1974.200),Ziel,POINT Z (2739850.838 1217402.568 1379.800),LINESTRING Z (2737832.731 1214786.689 1974.200...


In [14]:
gdf_calc = calc_leistungskm(gpx)

In [19]:
print(gdf_calc)
print(type(gdf_calc))


(   segment_id von_pkt_name                                von_pkt_geom  \
0           1   Startpunkt  POINT Z (2737832.731 1214786.689 1974.200)   

  bis_pkt_name                                bis_pkt_geom  \
0         Ziel  POINT Z (2739850.838 1217402.568 1379.800)   

                                        segment_geom  cumulative_km  \
0  LINESTRING Z (2737832.731 1214786.689 1974.200...       14.80357   

   elevation  Leistungskm [km]  Marschzeit [min]  
0     -594.4          41.39157             621.0  , 14.804, 1032.0, -1627.0, 10, 0)
<class 'tuple'>


In [28]:
df = gdf_calc[0]  # get the DataFrame from the tuple

gdf_show = pd.DataFrame({
    'Abschnitt': df['segment_id'],
    'Von': df['von_pkt_name'],
    'Nach': df['bis_pkt_name'],
    'Laenge [km]': df['cumulative_km'].round(2),
    'Hoehenmeter [m]': df['elevation'].round(1),
    'Leistungskm': df['Leistungskm [km]'].round(2),
    'Marschzeit [min]': df['Marschzeit [min]'].round(0),
    'Hinweis': ""  # Add an empty 'Hinweis' column
})


gdf_show.head()


,Abschnitt,Von,Nach,Laenge [km],Hoehenmeter [m],Leistungskm,Marschzeit [min],Hinweis
0,1,Startpunkt,Ziel,14.8,-594.4,41.39,621.0,
